<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/main/LibraryModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 설치 및 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# 트랜스포머 설치
!pip install transformers
from transformers import AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook


''' 사용예시: 이런식으로 사용하여 TPU로 모델 학습 가능
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
'''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-knpyho_1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-knpyho_1


" 사용예시: 이런식으로 사용하여 TPU로 모델 학습 가능\nresolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])\ntf.config.experimental_connect_to_cluster(resolver)\ntf.tpu.experimental.initialize_tpu_system(resolver)\nstrategy = tf.distribute.TPUStrategy(resolver)\ndef create_model():\n  return tf.keras.Sequential(\n      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),\n       tf.keras.layers.Conv2D(256, 3, activation='relu'),\n       tf.keras.layers.Flatten(),\n       tf.keras.layers.Dense(256, activation='relu'),\n       tf.keras.layers.Dense(128, activation='relu'),\n       tf.keras.layers.Dense(10)])\n\nwith strategy.scope():\n  model = create_model()\n  model.compile(optimizer='adam',\n                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),\n                metrics=['sparse_categorical_accuracy'])\n"

In [ ]:
#KoBERT 
#https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb#scrollTo=jlj7KeoNV5yc
#위는 KoBERT 사용 네이버 리뷰 분석
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
kr_tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
kr_model, kr_vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [ ]:
#EnBERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
# Load pretrained model/tokenizer
en_tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
en_model = model_class.from_pretrained(pretrained_weights)

## 데이터 가공 파트


In [45]:
#데이터 가공
#청구기호 숫자 앞 2개만 따와서 각각 매핑. 65~70은 예외적으로 경영학
data = pd.read_csv("/content/gdrive/MyDrive/LibraryCsv/library1.csv")
data = data.drop(['저자', '발행사항'], axis=1)
data = data.dropna()
data

,제목,청구번호
0,La place,[ 843.914 E71p ]
1,Neurobiology of disease,[ 612.8 G487n2 ]
2,Stochastic processes in engineering systems,[ 519.202462 W872ss ]
3,China : a new history,[ 952 F164c2 ]
4,Technical writing for dummies,[ 808.0666 L751t ]
...,...,...
100095,Surfactant solutions : new methods of investig...,[ 668.1 S961M V.22 ]
100096,Study guide to accompany Schall and Haley intr...,[ 658.15 S862IS2 ]
100097,Introduction to financial management,[ 658.15 S862I2 ]
100098,An introduction to financial management,[ 658.15 S689I2 ]


In [46]:
import re 
result = re.search('\s[0-9]{2}',data['청구번호'][0])
data['청구번호'] = data['청구번호'].replace({r'\[ (. )?(\d{2})\d.*' : r'\2'}, regex=True)
data #REGEX 로 가공!

,제목,청구번호
0,La place,84
1,Neurobiology of disease,61
2,Stochastic processes in engineering systems,51
3,China : a new history,95
4,Technical writing for dummies,80
...,...,...
100095,Surfactant solutions : new methods of investig...,66
100096,Study guide to accompany Schall and Haley intr...,65
100097,Introduction to financial management,65
100098,An introduction to financial management,65


In [47]:
data['청구번호'] = pd.to_numeric(data['청구번호'])
data.loc[(data['청구번호'] < 10), ['청구번호']] = 1000
data.loc[(data['청구번호'] < 20), ['청구번호']] = 1001
data.loc[(data['청구번호'] < 30), ['청구번호']] = 1002
data.loc[(data['청구번호'] < 40), ['청구번호']] = 1003
data.loc[(data['청구번호'] < 50), ['청구번호']] = 1004
data.loc[(data['청구번호'] < 60), ['청구번호']] = 1005
data.loc[(data['청구번호'] < 65), ['청구번호']] = 1006 #경영학쪽은 따로 분류이기 떄문에 65 사용
data.loc[(data['청구번호'] < 70), ['청구번호']] = 1007 #즉, 1007 쪽이 경영학책
data.loc[(data['청구번호'] < 80), ['청구번호']] = 1008
data.loc[(data['청구번호'] < 90), ['청구번호']] = 1009
data.loc[(data['청구번호'] <  100), ['청구번호']] = 1010

labels = {'0':'총류',
          '1':'철학',
          '2':'종교',
          '3':'사회학',
          '4':'언어',
          '5':'자연과학',
          '6':'기술과학',
          '7':'경영학',
          '8':'예술',
          '9':'문학',
          '10':'역사'
          }

In [48]:
data['청구번호'] = data['청구번호']%1000
data['청구번호'] = data['청구번호'].astype(int)
data['청구번호'] = data['청구번호'].apply(str)
data['청구번호'] = data['청구번호'].replace(labels)
data

,제목,청구번호
0,La place,문학
1,Neurobiology of disease,기술과학
2,Stochastic processes in engineering systems,자연과학
3,China : a new history,역사
4,Technical writing for dummies,문학
...,...,...
100095,Surfactant solutions : new methods of investig...,경영학
100096,Study guide to accompany Schall and Haley intr...,경영학
100097,Introduction to financial management,경영학
100098,An introduction to financial management,경영학


In [8]:
#input_string = "Peace-building and development in Guatemala and Northern Ireland"
regex = '.*[ㄱ-ㅎ|ㅏ-ㅣ|가-힣].*'
kor_data = data[data.제목.str.match(regex)]                                 # 한글이 있다면 따로 빼내기
eng_data = pd.concat([data, kor_data, kor_data]).drop_duplicates(keep=False)

kor_data

,제목,청구번호
2374,보기왕이 온다 :사와무라 이치 장편소설,9
2496,레드 팀을 만들어라 :가장 뛰어난 답을 얻는 리더의 비밀,10
2793,"커피 드림 :꿈꾸는 커피 회사, 이디야 이야기",9
3431,알라 :기독교와 이슬람의 신은 같은가?,1
3653,천천히 읽기 그리고 생각하기 :뜨거운 울림을 위한 독서 이야기,9
...,...,...
99429,Economic growth and agriculture = 經濟成長과 農業 - T...,1
99430,Economic theory and underdeveolped regions = =...,1
99431,The stages of economic growth = 經濟成長의 諸段階 - W....,1
99449,강한회사는 무엇이 다른가,9


## Tokenization 과정

In [9]:
from tokenizers import Tokenizer
class Dataset(Dataset, Tokenizer):
    def __init__(self, df, tokenizer):
        self.labels = [labels[label] for label in df['청구번호']]
        self.texts = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['text']]
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[isc]
        return text, label

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)
train_dataset = Dataset(eng_data)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

RuntimeError: ignored

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for text, label in enumerate(tqdm(train_loader, 0)):
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1